In [1]:
import time
import requests
import io
import os
import time 
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import quote_plus  
from tqdm import tqdm  
import pickle



In [2]:
def get_google_images_url(query):
    # Encode the query to ensure it's URL-safe
    encoded_query = quote_plus(query)
    return f"https://www.google.com/search?hl=en&tbm=isch&q={encoded_query}"

In [3]:
def get_imgs_from_google(driver, max_num_imgs, url):
    driver.get(url)
    img_urls = set()
    try:
        driver.find_element(By.ID, 'W0wltc').click()
    except Exception as e:
        print("Error clicking consent button:", e)

    # Set up the progress bar with max_num_imgs as the total number of iterations
    with tqdm(total=max_num_imgs, desc="Collecting Images") as pbar:
        while len(img_urls) < max_num_imgs:
            imgs = driver.find_elements(By.CLASS_NAME, "mNsIhb")
            for i, img in enumerate(imgs[len(img_urls):], start=len(img_urls)):
                actions = ActionChains(driver)
                actions.move_to_element(img).perform()
                time.sleep(2)  # Wait for overlay elements to load
                driver.execute_script("arguments[0].click();", img)
                try:
                    images = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".sFlh5c.FyHeAf.iPVvYb"))
                    )
                    img_url = images.get_attribute("src")
                    if img_url not in img_urls:  # Avoid duplicates
                        img_urls.add(img_url)
                        pbar.update(1)  # Update the progress bar
                except Exception as e:
                    continue
                if len(img_urls) >= max_num_imgs:
                    break

    driver.quit()
    return img_urls

In [4]:
def download_image(download_path, urls, prefix):
    if not os.path.exists(download_path):
        os.makedirs(download_path)  # Create the directory if it doesn't exist
    
    # Use tqdm for a progress bar over the range of URLs
    for i, url in enumerate(tqdm(urls, desc="Downloading Images", unit="image")):
        try:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            
            # File path with prefix and image number
            file_path = os.path.join(download_path, f"{prefix}_{i + 1}.jpg")
            
            # Convert image to RGB if it's in RGBA or P mode
            if image.mode in ("RGBA", "P"):
                image = image.convert("RGB")
            
            # Save the image as JPEG
            with open(file_path, "wb") as f:
                image.save(f, "JPEG")
            # print(f"{i + 1} --> done")  # Progress message

        except (requests.exceptions.RequestException, IOError) as e:
        # ``Log and continue on errors without breaking the loop
            continue


In [37]:
driver = webdriver.Chrome()  
search ="robots pictures"
url_robots = get_google_images_url(search)
urls_robots = get_imgs_from_google(driver,300,url_robots)
driver.quit()

In [38]:

# Save the URLs array to a file using pickle
with open("urls_robots.pkl", "wb") as file:
    pickle.dump(urls_robots, file)
    print("URLs saved to urls_robots.pkl")


URLs saved to urls_robots.pkl


In [5]:
driver = webdriver.Chrome()  
search ="people photography"
url_humans = get_google_images_url(search)
urls_humans = get_imgs_from_google(driver,300,url_humans)
driver.quit()

In [6]:
with open("urls_humans.pkl", "wb") as file:
    pickle.dump(urls_humans, file)
    print("URLs saved to urls_humans.pkl")

URLs saved to urls_humans.pkl


In [9]:
# Load the URLs array from the pickle file
with open("urls_robots.pkl", "rb") as file:
    urls_robots = pickle.load(file)
    print("URLs loaded from urls_robots.pkl")


URLs loaded from urls_robots.pkl


In [7]:
# Load the URLs array from the pickle file
with open("urls_humans.pkl", "rb") as file:
    urls_robots = pickle.load(file)
    print("URLs loaded from urls_humans.pkl")


URLs loaded from urls_humans.pkl


In [8]:
# download_image("images//Robots",urls_robots,"robot")
download_image("images//Humans",urls_humans,"human")

In [10]:
download_image("images//Robots",urls_robots,"robot")


  warnings.warn(
